# 05-04: AgentCore Runtime Evaluations

## Overview

This notebook demonstrates how to deploy a Strands-based AI agent to **AgentCore Runtime** and evaluate its performance using the native **AgentCore Evaluations** API with built-in evaluators.

### What is AgentCore Runtime?

**AgentCore Runtime** is an AWS managed service for deploying and running AI agents with built-in observability. It provides:

- **Managed Infrastructure**: Deploy agents without managing servers or containers directly
- **Built-in Observability**: Automatic logging of agent execution traces to CloudWatch
- **Scalable Invocation**: Invoke agents via API with session management
- **Easy Deployment**: Use the `bedrock-agentcore-starter-toolkit` for streamlined deployment workflows

### What is AgentCore Evaluations?

**AgentCore Evaluations** is an AWS service that provides the `evaluate()` API for assessing agent performance using built-in evaluators. Key features include:

- **Built-in Evaluators**: Pre-configured evaluators like `Builtin.Helpfulness` and `Builtin.ToolSelectionAccuracy`
- **Session Span Analysis**: Evaluate agent behavior based on execution traces from CloudWatch logs
- **Objective Scoring**: Get numeric scores, labels, and detailed explanations for each evaluation
- **Token Usage Tracking**: Monitor the cost of running evaluations

### Why Use AgentCore Runtime + Evaluations?

This combination provides an end-to-end workflow for deploying and assessing AI agents:

1. **Deploy Once, Evaluate Continuously**: Deploy your agent to a managed runtime and run evaluations against real execution traces
2. **Production-Ready Observability**: Leverage CloudWatch integration for comprehensive logging
3. **Standardized Evaluation**: Use AWS-provided evaluators for consistent, objective assessment
4. **Iterative Improvement**: Use evaluation results to identify areas for agent improvement

## What You'll Learn

In this module, you will learn how to:

1. **Create a Strands Agent**: Build a city search agent using the `strands-agents` framework with web search capabilities

2. **Deploy to AgentCore Runtime**: Use the `bedrock-agentcore-starter-toolkit` to deploy your agent to AWS managed infrastructure

3. **Invoke the Deployed Agent**: Call your agent via the `invoke_agent_runtime` API with session management

4. **Retrieve Session Spans**: Extract execution traces from CloudWatch logs for evaluation

5. **Run AgentCore Evaluations**: Use the `evaluate()` API with built-in evaluators:
   - `Builtin.Helpfulness` - Assess response quality and usefulness
   - `Builtin.ToolSelectionAccuracy` - Evaluate whether the agent used appropriate tools

6. **Analyze Evaluation Results**: Interpret scores, labels, and explanations to understand agent performance

7. **Clean Up Resources**: Properly destroy deployed agents to avoid unnecessary AWS charges

### Prerequisites

Before running this notebook, ensure you have:

- AWS credentials configured with appropriate permissions for Bedrock, AgentCore, and CloudWatch
- Access to Amazon Bedrock foundation models (Nova Micro)
- Python 3.9+ environment

### Module Workflow

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  1. Create      │     │  2. Deploy to   │     │  3. Invoke      │
│  Strands Agent  │ ──▶ │  AgentCore      │ ──▶ │  Agent          │
│                 │     │  Runtime        │     │                 │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                                                        │
                                                        ▼
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  6. Analyze     │     │  5. Run         │     │  4. Retrieve    │
│  Results        │ ◀── │  Evaluations    │ ◀── │  Session Spans  │
│                 │     │                 │     │  from CloudWatch│
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

## Section 1: Dependencies Installation

First, we'll install all the required packages for this workshop module. This includes:

- **strands-agents** and **strands-agents-tools**: Framework for building AI agents with tool capabilities
- **bedrock-agentcore** and **bedrock-agentcore-starter-toolkit**: AWS packages for AgentCore Runtime deployment and Evaluations API
- **ddgs**: DuckDuckGo search library for web search functionality
- **boto3**: AWS SDK for Python
- **pandas**: Data manipulation library for handling test cases

In [1]:
# Install required packages
# Using -q flag for quiet output to reduce notebook clutter

%pip install -q strands-agents strands-agents-tools
%pip install -q bedrock-agentcore bedrock-agentcore-starter-toolkit
%pip install -q ddgs
%pip install -q boto3 pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import sys

# Strands agents framework
from strands import Agent, tool
from strands.models.bedrock import BedrockModel

# AgentCore packages
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore_starter_toolkit import Runtime

# DuckDuckGo search
from ddgs import DDGS

# Web page fetching
import requests
from bs4 import BeautifulSoup

# AWS and data handling
import boto3
import pandas as pd

# Additional standard library imports we'll need
import json
import uuid
import time
from datetime import datetime

## Section 2: City Search Agent Creation

In this section, we'll create a city search agent that can answer questions about city populations and areas. The agent uses:

- **Web Search Tool**: A custom tool using DuckDuckGo to search for current information
- **Get Page Tool**: A tool to fetch and parse web page content for detailed information
- **BedrockModel**: Amazon Nova Micro for optimized latency and cost
- **System Prompt**: Instructions for the agent to act as a tour guide with structured XML output

### Tool Definitions

We'll start by defining the `web_search` and `get_page` tools using the `@tool` decorator from Strands. The `web_search` tool queries DuckDuckGo and returns the top 5 search results, while `get_page` fetches and extracts text content from URLs.

In [11]:
@tool
def web_search(topic: str) -> str:
    """
    Search DuckDuckGo for information about a topic.
    
    Args:
        topic: Search query string
    
    Returns:
        String containing top 5 search results with titles, URLs, and snippets
    """
    try:
        # Use DuckDuckGo search to find information
        results = DDGS().text(topic, max_results=5)
        
        if not results:
            return "No search results found"
        
        # Format results as a readable string
        result_string = ""
        for i, result in enumerate(results):
            title = result.get('title', 'No title')
            url = result.get('href', 'No URL')
            snippet = result.get('body', 'No description')
            result_string += f"Result {i+1}: {title}\nURL: {url}\nSnippet: {snippet}\n\n"
        
        return result_string
        
    except Exception as e:
        return f"Search error: {str(e)}"

@tool
def get_page(url: str) -> str:
    """
    Fetch and return the raw text content from a URL.
    
    Args:
        url: The URL to fetch content from
    
    Returns:
        String containing the text content of the web page
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        bs = BeautifulSoup(response.text, 'html.parser')
        return bs.text
    except Exception as e:
        return f"Error fetching page: {str(e)}"

# Verify the tools are defined correctly
print(f"Tool name: {web_search.tool_name}")
print(f"Tool description: {web_search.tool_spec.get('description', 'No description')}")

print(f"\nTool name: {get_page.tool_name}")
print(f"Tool description: {get_page.tool_spec.get('description', 'No description')}")

Tool name: web_search
Tool description: Search DuckDuckGo for information about a topic.

Returns:
    String containing top 5 search results with titles, URLs, and snippets

Tool name: get_page
Tool description: Fetch and return the raw text content from a URL.

Returns:
    String containing the text content of the web page


### Model Configuration

Next, we configure the BedrockModel with Amazon Nova Micro. We choose Nova Micro for this workshop because:

- **Optimized Latency**: Nova Micro provides fast response times, ideal for interactive agent workflows
- **Cost Efficiency**: Lower cost per token compared to larger models, suitable for evaluation scenarios with multiple invocations
- **Sufficient Capability**: Handles city search queries effectively while keeping costs manageable

We also configure timeout settings to ensure reliable API calls during evaluation runs.

In [12]:
from botocore.config import Config

# Configure timeout settings for Bedrock API calls
# These settings ensure reliable operation during evaluation scenarios:
# - connect_timeout: Maximum time to establish connection (5 seconds)
# - read_timeout: Maximum time to wait for response (60 seconds for agent reasoning)
# - retries: Single retry attempt to handle transient failures
bedrock_config = Config(
    connect_timeout=5,
    read_timeout=60,
    retries={"max_attempts": 1}
)

# Configure BedrockModel with Amazon Nova Micro
# Model choice rationale:
# - us.amazon.nova-micro-v1:0 provides optimized latency and cost for agent workflows
# - Suitable for city search queries that require web search tool usage
# - Cross-region inference prefix (us.) enables automatic region routing
model_id = "us.amazon.nova-micro-v1:0"

city_search_model = BedrockModel(
    model_id=model_id,
    boto_client_config=bedrock_config
)

### Agent Creation with System Prompt

Now we create the City Search Agent by combining:

1. **System Prompt**: Instructions that define the agent's persona and behavior
   - Acts as a helpful tour guide
   - Uses tools to retrieve current data
   - Outputs structured XML tags for programmatic parsing

2. **Tools**: The `web_search` tool for retrieving city information

3. **Model**: The BedrockModel configured with Amazon Nova Micro

#### XML Output Format

The agent is instructed to output responses with specific XML tags:
- `<response>`: Human-friendly response with context and fun facts
- `<pop>`: Population value (numbers only, no commas)
- `<area>`: Area value in square miles (numbers only)

This structured output format enables programmatic parsing of the agent's responses for evaluation and downstream processing.

In [13]:
# Define the system prompt for the City Search Agent
# This prompt establishes:
# 1. Tour guide persona - friendly, informative responses
# 2. Tool usage requirement - always use tools for current data
# 3. XML output format - structured tags for programmatic parsing

system_prompt = '''You are a helpful tour guide. Customers may ask you about the population and size of cities.
You should use tools to retrieve all data, to make sure it is as current as possible.
Please put your human friendly response in 'response' XML tags, and then follow with your data results in 'pop' and 'area' XML tags, for programatic processing.
The values in the 'pop' and 'area' XML tags should only be numbers, no words or commas.
You may also include a fun fact about the city in question.
For example, your response to "what is the population and size of Somewhereland?" might be:
<response>Somewhereland is a great place to get a hot dog. It has a population of 3000 people, and is 100 square miles.</response>
<pop>3000</pop><area>100</area>
'''

# Create the City Search Agent
# Combining the system prompt, tools, and BedrockModel
city_search_agent = Agent(
    system_prompt=system_prompt,
    tools=[web_search, get_page],
    model=city_search_model
)

print("✓ City Search Agent created successfully!")
print("\nAgent Configuration:")
print(f"  Model: {model_id}")
print(f"  Tools: ['web_search', 'get_page']")
print(f"  System prompt length: {len(system_prompt)} characters")
print("\nExpected output format:")
print("  <response>Human-friendly response with fun facts</response>")
print("  <pop>numeric_population</pop>")
print("  <area>numeric_area</area>")

✓ City Search Agent created successfully!

Agent Configuration:
  Model: us.amazon.nova-micro-v1:0
  Tools: ['web_search', 'get_page']
  System prompt length: 743 characters

Expected output format:
  <response>Human-friendly response with fun facts</response>
  <pop>numeric_population</pop>
  <area>numeric_area</area>


### Test Agent Invocation

Before deploying to AgentCore Runtime, let's verify that our agent works correctly by testing it with a sample city query.

This test invocation will:
1. Send a query asking about Seattle, WA's population and area
2. Display the full response from the agent
3. Show the XML tags (`<response>`, `<pop>`, `<area>`) in the output

A successful test confirms that:
- The agent can use the `web_search` tool to retrieve information
- The response includes the expected XML tag structure
- The model is properly configured and accessible

In [14]:
# Test the City Search Agent with a sample query
# This verifies the agent works correctly before deployment to AgentCore Runtime

# Sample query about Seattle, WA
test_query = "What is the population and area of Seattle, WA?"

print("Testing City Search Agent...")
print(f"Query: {test_query}")
print("\n" + "="*60)
print("Agent Response:")
print("="*60 + "\n")

# Invoke the agent with the test query
test_response = city_search_agent(test_query)

# Display the full response
print(test_response)

print("\n" + "="*60)
print("Test Complete!")
print("="*60)

# Verify XML tags are present in the response
response_str = str(test_response)
xml_tags_present = {
    "<response>": "<response>" in response_str,
    "<pop>": "<pop>" in response_str,
    "<area>": "<area>" in response_str
}

print("\nXML Tag Verification:")
for tag, present in xml_tags_present.items():
    status = "✓" if present else "✗"
    print(f"  {status} {tag} tag {'found' if present else 'NOT found'}")

if all(xml_tags_present.values()):
    print("\n✓ Agent test passed! All expected XML tags are present.")
else:
    print("\n⚠ Warning: Some expected XML tags are missing from the response.")

Testing City Search Agent...
Query: What is the population and area of Seattle, WA?

Agent Response:

<thinking> To provide the population and area of Seattle, WA, I will use the web_search tool to find reliable sources of information. The search query will be "Seattle WA population and area". After obtaining the results, I will extract the relevant data from the top sources.</thinking>

Tool #1: web_search
<response>Seattle, WA is a vibrant city with a lot to offer. As of the most recent data from 2020, the population of Seattle is 740,600. The city covers an area of approximately 217 square miles. It's interesting to note that Seattle is known for its coffee culture, and it's the birthplace of Starbucks!</response>
<pop>740600</pop><area>217</area><response>Seattle, WA is a vibrant city with a lot to offer. As of the most recent data from 2020, the population of Seattle is 740,600. The city covers an area of approximately 217 square miles. It's interesting to note that Seattle is kno

## Section 3: AgentCore Runtime Deployment

Now that we've verified our City Search Agent works correctly, we'll deploy it to **AgentCore Runtime**. This deployment process involves:

1. **Create Agent File**: Write a Python file (`citysearch.py`) that wraps our agent with `BedrockAgentCoreApp` integration
2. **Configure Deployment**: Use the `bedrock-agentcore-starter-toolkit` to set up deployment configuration
3. **Launch to Runtime**: Deploy the containerized agent to AgentCore Runtime
4. **Verify Deployment**: Confirm the agent is ready and test invocation via the API

### Why Deploy to AgentCore Runtime?

Deploying to AgentCore Runtime provides several benefits:

- **Managed Infrastructure**: No need to manage servers, containers, or scaling
- **Built-in Observability**: Automatic logging of all agent interactions to CloudWatch
- **Session Management**: Track conversations across multiple invocations
- **Evaluation Integration**: Session spans can be used directly with AgentCore Evaluations API

### Agent File Structure

The `citysearch.py` file follows the AgentCore Runtime pattern:

```python
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

@app.entrypoint
def invoke(payload):
    # Process the request and return response
    pass

if __name__ == "__main__":
    app.run()
```

This pattern enables AgentCore Runtime to:
- Receive incoming requests via the `invoke` function
- Automatically instrument the agent for observability
- Manage the agent lifecycle in a containerized environment

In [24]:
%%writefile ./citysearch.py
# City Search Agent for AgentCore Runtime Deployment
# This file creates a deployable agent using BedrockAgentCoreApp
#
# The agent:
# - Uses DuckDuckGo web search to find city information
# - Can fetch and parse web pages for detailed information
# - Returns responses with XML tags for programmatic parsing
# - Is configured for AgentCore Runtime with the @app.entrypoint decorator

from botocore.config import Config
from ddgs import DDGS
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from boto3.session import Session
import requests
from bs4 import BeautifulSoup

# Get the current AWS region from the boto session
boto_session = Session()
region = boto_session.region_name

# Configure timeout settings for Bedrock API calls
# - connect_timeout: 5 seconds to establish connection
# - read_timeout: 20 seconds for response (optimized for fast queries)
# - retries: Disabled to fail fast and avoid hanging
quick_config = Config(
    connect_timeout=5,
    read_timeout=20,
    retries={"max_attempts": 0}
)


@tool
def web_search(topic: str) -> str:
    """
    Search DuckDuckGo for information about a topic.
    
    Args:
        topic: Search query string
    
    Returns:
        String containing top 5 search results with titles, URLs, and snippets
    """
    try:
        results = DDGS().text(topic, region=region, max_results=5)
        return results if results else "No results found."
    except Exception as e:
        return f"Search error: {str(e)}"


@tool
def get_page(url: str) -> str:
    """
    Fetch and return the raw text content from a URL.
    
    Args:
        url: The URL to fetch content from
    
    Returns:
        String containing the text content of the web page
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        bs = BeautifulSoup(response.text, 'html.parser')
        return bs.text
    except Exception as e:
        return f"Error fetching page: {str(e)}"


# Configure BedrockModel with Amazon Nova Micro
# Nova Micro is chosen for optimized latency and cost in agent workflows
chatbot_model_name = "us.amazon.nova-micro-v1:0"
chatbot_model = BedrockModel(
    model_id=chatbot_model_name,
    boto_client_config=quick_config
)

# Define the system prompt for the City Search Agent
# This establishes the tour guide persona and XML output format
system_prompt = '''You are a helpful tour guide. Customers may ask you about the population and size of cities.
You should use tools to retrieve all data, to make sure it is as current as possible.
Please put your human friendly response in 'response' XML tags, and then follow with your data results in 'pop' and 'area' XML tags, for programatic processing.
The values in the 'pop' and 'area' XML tags should only be numbers, no words or commas.
You may also include a fun fact about the city in question.
For example, your response to "what is the population and size of Somewhereland?" might be:
<response>Somewhereland is a great place to get a hot dog. It has a population of 3000 people, and is 100 square miles.</response>
<pop>3000</pop><area>100</area>
'''

# Create the City Search Agent with web_search and get_page tools
chatbot = Agent(
    system_prompt=system_prompt,
    tools=[web_search, get_page],
    model=chatbot_model
)

# Initialize the AgentCore Runtime App
# This enables deployment to AgentCore Runtime with built-in observability
app = BedrockAgentCoreApp()


@app.entrypoint
def invoke(payload):
    """
    AgentCore Runtime entrypoint function.
    
    This function is called by AgentCore Runtime when the agent receives a request.
    It extracts the prompt from the payload, invokes the agent, and returns the response.
    
    Args:
        payload: Dictionary containing 'prompt' key with user query
    
    Returns:
        Agent response string with XML tags for parsing
    """
    user_input = payload.get("prompt", "")
    
    # Invoke the agent with the user's query
    response = chatbot(user_input)
    
    # Extract and return the text content from the response
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    # Run the AgentCore Runtime app
    # This starts the agent server when running locally or in a container
    app.run()

Overwriting ./citysearch.py


In [25]:
# Verify the citysearch.py file was created successfully
import os

agent_file = "./citysearch.py"

if os.path.exists(agent_file):
    file_size = os.path.getsize(agent_file)
    print(f"✓ citysearch.py created successfully!")
    print(f"  File size: {file_size} bytes")
    print(f"  Location: {os.path.abspath(agent_file)}")

else:
    print(f"✗ Error: {agent_file} was not created")

✓ citysearch.py created successfully!
  File size: 4309 bytes
  Location: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/citysearch.py


### Configure AgentCore Runtime Deployment

Now we'll configure the deployment using the `bedrock-agentcore-starter-toolkit`. The `Runtime.configure()` method sets up all the necessary AWS resources and configuration for deploying our agent.

#### Configuration Parameters

| Parameter | Description |
|-----------|-------------|
| `entrypoint` | The Python file containing the agent code with `@app.entrypoint` decorator |
| `auto_create_execution_role` | Automatically create an IAM execution role for the agent |
| `auto_create_ecr` | Automatically create an ECR repository for the container image |
| `requirements_file` | Path to the requirements.txt file with dependencies |
| `region` | AWS region for deployment |
| `agent_name` | Name identifier for the deployed agent |

#### What Happens During Configuration

The `configure()` method:
1. Parses the entrypoint file to identify the `BedrockAgentCoreApp` configuration
2. Creates a `.bedrock_agentcore.yaml` configuration file
3. Generates a `Dockerfile` for containerizing the agent
4. Creates a `.dockerignore` file to exclude unnecessary files
5. Sets up IAM roles and ECR repository references (if auto-create is enabled)

After configuration, we'll be ready to launch the agent to AgentCore Runtime.

In [30]:
# Configure AgentCore Runtime deployment
# This sets up all necessary AWS resources and configuration files

from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

# Get the current AWS region from the boto session
boto_session = Session()
region = boto_session.region_name

# Define the agent name for deployment
# This name will be used to identify the agent in AgentCore Runtime
agent_name = "citysearch"

# Create a Runtime instance for managing the deployment
agentcore_runtime = Runtime()

# Configure the deployment with all required parameters
print(f"Configuring AgentCore Runtime deployment...")
print(f"  Agent name: {agent_name}")
print(f"  Region: {region}")
print(f"  Entrypoint: citysearch.py")
print(f"  Requirements: requirements.txt")
print()

configure_response = agentcore_runtime.configure(
    entrypoint="citysearch.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

# Display the configuration result
print("="*60)
print("Configuration Summary")
print("="*60)
print(f"\n✓ Configuration completed successfully!")
print(f"\nGenerated files:")
print(f"  - Config file: {configure_response.config_path}")
print(f"  - Dockerfile: {configure_response.dockerfile_path}")
print(f"  - Docker ignore: {configure_response.dockerignore_path}")
print(f"\nDeployment settings:")
print(f"  - Runtime: {configure_response.runtime}")
print(f"  - Region: {configure_response.region}")
print(f"  - Account ID: {configure_response.account_id}")
print(f"  - Auto-create ECR: {configure_response.auto_create_ecr}")
print(f"\nThe agent is now configured and ready for launch!")

# Display the full configuration response for reference
print("\n" + "="*60)
print("Full Configuration Response:")
print("="*60)
configure_response

Entrypoint parsed: file=/Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/citysearch.py, bedrock_agentcore_name=citysearch
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: citysearch


Configuring AgentCore Runtime deployment...
  Agent name: citysearch
  Region: us-east-1
  Entrypoint: citysearch.py
  Requirements: requirements.txt



💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
Network mode: PUBLIC


📄 Generated Dockerfile: 
/Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-R
untime-Evals/Dockerfile

Generated .dockerignore: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.dockerignore
Setting 'citysearch' as default agent
Bedrock AgentCore configured: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.bedrock_agentcore.yaml


Configuration Summary

✓ Configuration completed successfully!

Generated files:
  - Config file: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.bedrock_agentcore.yaml
  - Dockerfile: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/Dockerfile
  - Docker ignore: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.dockerignore

Deployment settings:
  - Runtime: None
  - Region: us-east-1
  - Account ID: 616455080294
  - Auto-create ECR: True

The agent is now configured and ready for launch!

Full Configuration Response:


ConfigureResult(config_path=PosixPath('/Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/Dockerfile'), dockerignore_path=PosixPath('/Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.dockerignore'), runtime='None', runtime_type=None, region='us-east-1', account_id='616455080294', execution_role=None, ecr_repository=None, auto_create_ecr=True, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

### Launch Agent to AgentCore Runtime

Now that the agent is configured, we'll launch it to AgentCore Runtime. The `runtime.launch()` method performs the following steps:

1. **Build Container**: Publishes the Docker image to ECR using CodeBuild (ARM64 architecture)
2. **Create/Update Agent**: Registers or updates the agent in AgentCore Runtime
3. **Deploy Endpoint**: Creates a runtime endpoint for invoking the agent

#### Launch Parameters

| Parameter | Description |
|-----------|-------------|
| `auto_update_on_conflict` | If `True`, automatically updates the agent if it already exists. If `False`, raises an error on conflict. |

#### What to Expect

The launch process typically takes **2-5 minutes** and includes:
- CodeBuild phases: QUEUED → PROVISIONING → DOWNLOAD_SOURCE → BUILD → POST_BUILD → FINALIZING → COMPLETED
- Agent deployment to AgentCore Runtime
- Endpoint creation and readiness polling

After launch completes, we'll store the **agent ARN** for use in subsequent evaluation steps. The agent ARN is required for:
- Invoking the agent via the `invoke_agent_runtime` API
- Constructing the CloudWatch log group name for span retrieval
- Running AgentCore Evaluations

In [31]:
# Launch the agent to AgentCore Runtime
# This step builds the container, deploys to AgentCore, and creates the endpoint
#
# auto_update_on_conflict=True ensures that if the agent already exists,
# it will be updated with the new configuration rather than raising an error

print("Launching agent to AgentCore Runtime...")
print("This process typically takes 2-5 minutes.")
print("\n" + "="*60)
print("Launch Progress:")
print("="*60 + "\n")

# Launch the agent with auto_update_on_conflict=True
# This will update the agent if it already exists
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

print("\n" + "="*60)
print("Launch Complete!")
print("="*60)

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'citysearch' to account 616455080294 (us-east-1)
Generated image tag: 20260129-174942-080
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: citysearch


Launching agent to AgentCore Runtime...
This process typically takes 2-5 minutes.

Launch Progress:



ECR repository available: 616455080294.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-citysearch
Getting or creating execution role for agent: citysearch
Using AWS region: us-east-1, account ID: 616455080294
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-672e22fb7a


✅ Reusing existing ECR repository: 616455080294.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-citysearch


✅ Reusing existing execution role: arn:aws:iam::616455080294:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-672e22fb7a
Execution role available: arn:aws:iam::616455080294:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-672e22fb7a
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: citysearch
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-672e22fb7a
Reusing existing CodeBuild execution role: arn:aws:iam::616455080294:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-672e22fb7a
Using dockerignore.template with 47 patterns for zip filtering
Uploaded source to S3: citysearch/source.zip
Updated CodeBuild project: bedrock-agentcore-citysearch-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 10.1s
🔄 DOWNLOAD_SOURCE started (total: 11s)
✅ DOWNLOAD_SOURCE comple


Launch Complete!


In [32]:
# Extract and store the agent ARN for later use
# The agent ARN is required for:
# - Invoking the agent via invoke_agent_runtime API
# - Constructing CloudWatch log group names
# - Running AgentCore Evaluations

# Extract the agent ARN from the launch result
agent_arn = launch_result.agent_arn

print("Agent ARN Extracted and Stored")
print("="*60)
print(f"\n✓ Agent ARN: {agent_arn}")

# Store the agent ARN using IPython's %store magic
# This persists the variable across notebook sessions
citysearch_agent_arn = agent_arn
%store citysearch_agent_arn

print(f"\n✓ Agent ARN stored as 'citysearch_agent_arn' for use in subsequent cells")

# Display the full launch result for reference
print("\n" + "="*60)
print("Full Launch Result:")
print("="*60)
print(f"\nAgent ARN: {launch_result.agent_arn}")
print(f"Endpoint ARN: {getattr(launch_result, 'endpoint_arn', 'N/A')}")
print(f"ECR Image: {getattr(launch_result, 'ecr_image', 'N/A')}")

# Display the launch result object
launch_result

Agent ARN Extracted and Stored

✓ Agent ARN: arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf
Stored 'citysearch_agent_arn' (str)

✓ Agent ARN stored as 'citysearch_agent_arn' for use in subsequent cells

Full Launch Result:

Agent ARN: arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf
Endpoint ARN: N/A
ECR Image: N/A


LaunchResult(mode='codebuild', tag='bedrock_agentcore-citysearch:None', env_vars=None, port=None, runtime=None, ecr_uri='616455080294.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-citysearch:20260129-174942-080', agent_id='citysearch-U6S1xN3zuf', agent_arn='arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf', codebuild_id='bedrock-agentcore-citysearch-builder:4f403444-d5c8-4806-97d6-aafa7899ddcd', build_output=None)

In [ ]:
# NOTE!  If you need to delete the local configuration, and start over from scratch, run this cell.
# This can happen if you deploy an agent then delete it, and the above cell is still trying to update the deleted agent.
# This is needed because in the configuration cell, we create local configuration files which are used by the launch cell.
if False:
    import os

    # Remove the config file so launch creates a new agent instead of updating
    config_files = ['.bedrock_agentcore.yaml', 'Dockerfile', '.dockerignore']
    for f in config_files:
        if os.path.exists(f):
            os.remove(f)
            print(f"Deleted {f}")
        else:
            print(f"{f} not found")

    print("\nNow re-run the configure() and launch() cells")

### Verify Agent Status

After launching the agent, we need to verify that it has reached the **READY** status before we can invoke it. The agent goes through several states during deployment:

| Status | Description |
|--------|-------------|
| `CREATING` | Agent is being created and deployed |
| `UPDATING` | Agent is being updated with new configuration |
| `READY` | Agent is ready to receive invocations |
| `CREATE_FAILED` | Agent creation failed |
| `UPDATE_FAILED` | Agent update failed |
| `DELETE_FAILED` | Agent deletion failed |

This cell polls the agent status every 10 seconds until it reaches a terminal state (READY or a failure state). A timeout of 5 minutes is enforced to prevent indefinite waiting.

**Note**: The agent must be in READY status before proceeding to the invocation and evaluation steps.

In [33]:
# Verify agent status is READY before proceeding
# This cell polls the agent status until it reaches a terminal state
#
# Terminal states:
# - READY: Agent is ready for invocations
# - CREATE_FAILED, UPDATE_FAILED, DELETE_FAILED: Deployment failed
#
# Timeout: 5 minutes (300 seconds) to prevent indefinite waiting

import time

# Configuration for status polling
POLL_INTERVAL_SECONDS = 10  # Time between status checks
TIMEOUT_SECONDS = 300       # Maximum wait time (5 minutes)

# Terminal states that indicate deployment is complete (success or failure)
TERMINAL_STATES = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

print("Verifying Agent Status")
print("="*60)
print(f"Polling interval: {POLL_INTERVAL_SECONDS} seconds")
print(f"Timeout: {TIMEOUT_SECONDS} seconds ({TIMEOUT_SECONDS // 60} minutes)")
print("\nPolling agent status...\n")

# Track elapsed time for timeout
start_time = time.time()
elapsed_time = 0

# Get initial status
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"[{elapsed_time:3d}s] Current status: {status}")

# Poll until terminal state or timeout
while status not in TERMINAL_STATES:
    # Check for timeout
    elapsed_time = int(time.time() - start_time)
    if elapsed_time >= TIMEOUT_SECONDS:
        print(f"\n" + "="*60)
        print("⚠ TIMEOUT: Agent did not reach READY status within 5 minutes")
        print("="*60)
        print(f"\nLast known status: {status}")
        print(f"Elapsed time: {elapsed_time} seconds")
        print("\nPossible actions:")
        print("  1. Wait longer and re-run this cell to check status again")
        print("  2. Check AWS Console for AgentCore Runtime deployment status")
        print("  3. Review CloudWatch logs for any deployment errors")
        break
    
    # Wait before next poll
    time.sleep(POLL_INTERVAL_SECONDS)
    
    # Get updated status
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    elapsed_time = int(time.time() - start_time)
    print(f"[{elapsed_time:3d}s] Current status: {status}")

# Display final result
print("\n" + "="*60)
print("Status Verification Complete")
print("="*60)

if status == 'READY':
    print(f"\n✓ Agent is READY!")
    print(f"  Total wait time: {elapsed_time} seconds")
    print(f"\nThe agent is now ready to receive invocations.")
elif status in ['CREATE_FAILED', 'UPDATE_FAILED', 'DELETE_FAILED']:
    print(f"\n✗ Deployment FAILED with status: {status}")
    print(f"  Total wait time: {elapsed_time} seconds")
    print(f"\nPlease check CloudWatch logs for error details.")
    print(f"You may need to fix the issue and re-run the launch cell.")
else:
    print(f"\n⚠ Status check ended with status: {status}")
    print(f"  This may indicate a timeout or unexpected state.")

# Display the final status
status

Verifying Agent Status
Polling interval: 10 seconds
Timeout: 300 seconds (5 minutes)

Polling agent status...



Retrieved Bedrock AgentCore status for: citysearch


[  0s] Current status: READY

Status Verification Complete

✓ Agent is READY!
  Total wait time: 0 seconds

The agent is now ready to receive invocations.


'READY'

### Test Invocation via invoke_agent_runtime API

Now that the agent is deployed and in READY status, we'll demonstrate a test invocation using the `invoke_agent_runtime` API. This API allows you to:

- **Invoke the deployed agent**: Send queries to your agent running on AgentCore Runtime
- **Manage sessions**: Use unique session IDs to track conversations and generate trace data
- **Receive responses**: Get the agent's response as a streaming body that can be parsed

#### API Parameters

| Parameter | Description |
|-----------|-------------|
| `agentRuntimeArn` | The ARN of the deployed agent (obtained from launch result) |
| `runtimeSessionId` | A unique session identifier (must be 33+ characters) |
| `payload` | JSON-encoded payload containing the prompt |
| `qualifier` | Optional qualifier (default: "DEFAULT") |

#### Session ID Requirements

The `runtimeSessionId` must be at least 33 characters long. We use a UUID-based format (`eval-session-{uuid}`) which ensures:
- Uniqueness across invocations
- Sufficient length (typically 49 characters)
- Easy identification in CloudWatch logs

This test invocation confirms that the deployment was successful and the agent can respond to queries via the API.

In [35]:
# Test invocation via invoke_agent_runtime API
# This demonstrates successful deployment by invoking the agent through the API
#
# The invoke_agent_runtime API:
# - Sends a query to the deployed agent
# - Returns a streaming response body
# - Generates trace data in CloudWatch for later evaluation

import boto3
import json
import uuid
import textwrap

# Create a boto3 client for bedrock-agentcore
# This client provides access to the invoke_agent_runtime API
agentcore_client = boto3.client('bedrock-agentcore', region_name=region)

# Generate a unique session ID (must be 33+ characters)
# Format: eval-session-{uuid} ensures uniqueness and sufficient length
test_session_id = f"eval-session-{uuid.uuid4()}"

# Prepare the test query payload
# The payload must be JSON-encoded with a 'prompt' key
test_query = "What is the population and area of Chicago, IL?"
payload = json.dumps({"prompt": test_query})

print("Test Invocation via invoke_agent_runtime API")
print("="*60)
print(f"\nAgent ARN: {citysearch_agent_arn}")
print(f"Session ID: {test_session_id}")
print(f"Session ID length: {len(test_session_id)} characters (minimum: 33)")
print(f"\nQuery: {test_query}")
print("\n" + "="*60)
print("Invoking agent...")
print("="*60 + "\n")

# Invoke the deployed agent using the invoke_agent_runtime API
response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=citysearch_agent_arn,
    runtimeSessionId=test_session_id,
    payload=payload,
    qualifier="DEFAULT"
)

# Read and parse the response
# The response body is a streaming object that needs to be read
response_body = response['response'].read()
response_data = json.loads(response_body)

# Display the response
print("Agent Response:")
print("-"*60)
if isinstance(response_data, str):
    # Response is a string, wrap it for readability
    wrapped = textwrap.fill(response_data, width=80)
    print(wrapped)
else:
    # Response is a dict/object, pretty print it
    formatted = json.dumps(response_data, indent=2)
    print(formatted)

print("\n" + "="*60)
print("Test Invocation Complete!")
print("="*60)

# Display response metadata
print(f"\nResponse Metadata:")
print(f"  HTTP Status Code: {response['ResponseMetadata']['HTTPStatusCode']}")
print(f"  Request ID: {response['ResponseMetadata']['RequestId']}")
print(f"  Runtime Session ID: {response.get('runtimeSessionId', 'N/A')}")
print(f"  Trace ID: {response.get('traceId', 'N/A')}")

# Verify XML tags are present in the response
response_str = str(response_data)
xml_tags_present = {
    "<response>": "<response>" in response_str,
    "<pop>": "<pop>" in response_str,
    "<area>": "<area>" in response_str
}

print("\nXML Tag Verification:")
for tag, present in xml_tags_present.items():
    status_icon = "✓" if present else "✗"
    print(f"  {status_icon} {tag} tag {'found' if present else 'NOT found'}")

if all(xml_tags_present.values()):
    print("\n✓ Deployment test PASSED! Agent is responding correctly via API.")
else:
    print("\n⚠ Warning: Some expected XML tags are missing from the response.")

print("\nThe agent is successfully deployed and responding to API invocations!")

Test Invocation via invoke_agent_runtime API

Agent ARN: arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf
Session ID: eval-session-29186765-44c7-4fab-ba3b-6ba32c2aa8cc
Session ID length: 49 characters (minimum: 33)

Query: What is the population and area of Chicago, IL?

Invoking agent...

Agent Response:
------------------------------------------------------------
<thinking> After reviewing the search results, I found several sources that
discuss Chicago's population, including projections and official data. The most
reliable source for the current population and area of Chicago is the City of
Chicago's official press release. Based on the latest data, Chicago has an
approximate population of 2.7 million people and covers an area of about 227.6
square miles. Here's the information I found:</thinking>  <response>Chicago is a
vibrant city with a rich history and diverse culture. As of the most recent
estimates, the population of Chicago is approximately 2.7

## Section 4: Agent Invocation for Trace Generation

Now that our agent is deployed and verified, we'll invoke it multiple times to generate trace data for evaluation. This section covers:

1. **Load Test Cases**: Read city data from `city_pop.csv` containing city, state, population, and area information
2. **Data Cleaning**: Remove Wikipedia footnotes (like `[c]`, `[d]`) from city names
3. **Generate Session IDs**: Create unique session identifiers for each invocation
4. **Execute Invocations**: Call the deployed agent for each test case
5. **Wait for Logs**: Allow time for CloudWatch log propagation

### Why Multiple Invocations?

Running multiple invocations with different test cases allows us to:

- **Generate Diverse Traces**: Capture agent behavior across different queries
- **Enable Statistical Analysis**: Calculate average scores across multiple evaluations
- **Test Consistency**: Verify the agent performs reliably across different inputs
- **Build Evaluation Dataset**: Create a representative sample for AgentCore Evaluations

### Test Case Data

The `city_pop.csv` file contains real city data with the following columns:

| Column | Description | Example |
|--------|-------------|--------|
| `city` | City name (may include Wikipedia footnotes) | "New York[c]" |
| `state` | State abbreviation | "NY" |
| `population` | City population | "8,478,072" |
| `land_area_mi2` | Land area in square miles | "300.5" |

In [36]:
# Load test cases from city_pop.csv
# This cell loads city data and prepares it for agent invocation
#
# Data cleaning steps:
# 1. Load the CSV file using pandas
# 2. Remove Wikipedia footnotes (e.g., [c], [d], [e]) from city names
# 3. Convert population strings to integers (remove commas)
# 4. Select the first 5 cities as test cases

import pandas as pd
import re

# Load the city population data from CSV
csv_path = "city_pop.csv"
print(f"Loading test cases from {csv_path}...")
print("="*60)

# Read the CSV file
city_df = pd.read_csv(csv_path)

print(f"\nTotal cities in dataset: {len(city_df)}")
print(f"Columns: {list(city_df.columns)}")

# Display raw data before cleaning (first 5 rows)
print("\n" + "="*60)
print("Raw Data (before cleaning):")
print("="*60)
print(city_df.head())

# Clean the data: Remove Wikipedia footnotes from city names
# Footnotes appear as [letter] at the end of city names, e.g., "New York[c]"
# Pattern: \[\w+\] matches [c], [d], [e], etc.
def clean_city_name(city_name):
    """Remove Wikipedia footnotes like [c], [d] from city names."""
    return re.sub(r'\[\w+\]', '', city_name).strip()

# Apply cleaning to the city column
city_df['city_clean'] = city_df['city'].apply(clean_city_name)

# Convert population to integer (remove commas and quotes)
def parse_population(pop_str):
    """Convert population string to integer, handling commas and quotes."""
    if isinstance(pop_str, str):
        # Remove commas and quotes
        clean_str = pop_str.replace(',', '').replace('"', '')
        return int(clean_str)
    return int(pop_str)

city_df['population_int'] = city_df['population'].apply(parse_population)

# Select the first 5 cities as test cases
NUM_TEST_CASES = 5
test_cases_df = city_df.head(NUM_TEST_CASES).copy()

# Create a clean display DataFrame with relevant columns
display_df = test_cases_df[['city_clean', 'state', 'population_int', 'land_area_mi2']].copy()
display_df.columns = ['City', 'State', 'Population', 'Area (sq mi)']

print("\n" + "="*60)
print(f"Test Cases (first {NUM_TEST_CASES} cities, cleaned):")
print("="*60)
print(display_df.to_string(index=False))

# Show which cities had footnotes removed
print("\n" + "="*60)
print("Data Cleaning Summary:")
print("="*60)
for idx, row in test_cases_df.iterrows():
    original = row['city']
    cleaned = row['city_clean']
    if original != cleaned:
        print(f"  ✓ Cleaned: '{original}' → '{cleaned}'")
    else:
        print(f"  - No change: '{original}'")

print(f"\n✓ Loaded {NUM_TEST_CASES} test cases successfully!")
print(f"\nThese cities will be used to invoke the agent and generate traces for evaluation.")

Loading test cases from city_pop.csv...

Total cities in dataset: 346
Columns: ['city', 'state', 'population', 'land_area_mi2']

Raw Data (before cleaning):
          city state population land_area_mi2
0  New York[c]    NY  8,478,072         300.5
1  Los Angeles    CA  3,878,704         469.5
2      Chicago    IL  2,721,308         227.7
3      Houston    TX  2,390,125         640.4
4      Phoenix    AZ  1,673,164           518

Test Cases (first 5 cities, cleaned):
       City State  Population Area (sq mi)
   New York    NY     8478072        300.5
Los Angeles    CA     3878704        469.5
    Chicago    IL     2721308        227.7
    Houston    TX     2390125        640.4
    Phoenix    AZ     1673164          518

Data Cleaning Summary:
  ✓ Cleaned: 'New York[c]' → 'New York'
  - No change: 'Los Angeles'
  - No change: 'Chicago'
  - No change: 'Houston'
  - No change: 'Phoenix'

✓ Loaded 5 test cases successfully!

These cities will be used to invoke the agent and generate trace

### Generate Unique Session IDs

Before invoking the agent for each test case, we need to generate unique session IDs. Session IDs are critical for:

- **Trace Identification**: Each invocation's trace data is tagged with its session ID in CloudWatch
- **Span Retrieval**: We use session IDs to filter and retrieve specific execution spans
- **Evaluation Correlation**: Session IDs link invocations to their evaluation results

#### Session ID Requirements

AgentCore Runtime requires session IDs to be **at least 33 characters** long. Our format `eval-session-{uuid}` ensures:

| Component | Length | Example |
|-----------|--------|--------|
| Prefix | 13 chars | `eval-session-` |
| UUID | 36 chars | `550e8400-e29b-41d4-a716-446655440000` |
| **Total** | **49 chars** | `eval-session-550e8400-e29b-41d4-a716-446655440000` |

#### Why UUID?

Using UUID (Universally Unique Identifier) for session IDs provides:

- **Guaranteed Uniqueness**: UUIDs are designed to be globally unique
- **No Coordination Required**: Can generate IDs independently without checking for collisions
- **Standard Format**: Well-understood format that's easy to parse and log
- **Sufficient Entropy**: 128 bits of randomness prevents accidental collisions

In [37]:
# Generate unique session IDs for each test case
# Session IDs are used to:
# 1. Track each invocation in CloudWatch logs
# 2. Retrieve session spans for evaluation
# 3. Correlate invocations with evaluation results
#
# Format: eval-session-{uuid}
# - Prefix 'eval-session-' identifies evaluation invocations
# - UUID ensures uniqueness (36 chars)
# - Total length: 49 characters (exceeds 33 char minimum)

import uuid

# Minimum session ID length required by AgentCore Runtime
MIN_SESSION_ID_LENGTH = 33

# Generate a unique session ID for each test case
# Store in a dictionary mapping city name to session ID for easy retrieval
session_ids = {}

# Also store as a list for ordered access
session_id_list = []

print("Generating Unique Session IDs")
print("="*60)
print(f"\nFormat: eval-session-{{uuid}}")
print(f"Minimum required length: {MIN_SESSION_ID_LENGTH} characters")
print("\n" + "-"*60)

for idx, row in test_cases_df.iterrows():
    city = row['city_clean']
    state = row['state']
    
    # Generate a unique session ID using UUID
    session_id = f"eval-session-{uuid.uuid4()}"
    
    # Store the session ID in both dictionary and list
    city_key = f"{city}, {state}"
    session_ids[city_key] = session_id
    session_id_list.append({
        'city': city,
        'state': state,
        'city_key': city_key,
        'session_id': session_id
    })
    
    # Display the generated session ID
    print(f"\n{idx + 1}. {city_key}")
    print(f"   Session ID: {session_id}")
    print(f"   Length: {len(session_id)} characters ✓")

print("\n" + "="*60)
print("Session ID Generation Summary")
print("="*60)
print(f"\n✓ Generated {len(session_ids)} unique session IDs")
print(f"✓ All session IDs meet minimum length requirement ({MIN_SESSION_ID_LENGTH}+ chars)")
print(f"✓ Session IDs stored in 'session_ids' dict and 'session_id_list' for retrieval")

# Verify all session IDs are unique
unique_ids = set(session_ids.values())
if len(unique_ids) == len(session_ids):
    print(f"✓ All session IDs are unique (no duplicates)")
else:
    print(f"⚠ Warning: Found duplicate session IDs!")

# Verify all session IDs meet minimum length
all_valid_length = all(len(sid) >= MIN_SESSION_ID_LENGTH for sid in session_ids.values())
if all_valid_length:
    print(f"✓ All session IDs are {MIN_SESSION_ID_LENGTH}+ characters")
else:
    print(f"⚠ Warning: Some session IDs are too short!")

# Display the session IDs dictionary for reference
print("\n" + "="*60)
print("Session IDs Dictionary (for later retrieval):")
print("="*60)
for city_key, sid in session_ids.items():
    print(f"  '{city_key}': '{sid}'")

Generating Unique Session IDs

Format: eval-session-{uuid}
Minimum required length: 33 characters

------------------------------------------------------------

1. New York, NY
   Session ID: eval-session-5c8d51fb-d526-4d68-b531-f2af5d3c2331
   Length: 49 characters ✓

2. Los Angeles, CA
   Session ID: eval-session-968acf9f-cc68-472c-b889-0b97dc5baacc
   Length: 49 characters ✓

3. Chicago, IL
   Session ID: eval-session-37bd25d6-30dc-4aa2-9528-4a1fa4a9f845
   Length: 49 characters ✓

4. Houston, TX
   Session ID: eval-session-900c5e48-9b66-42be-b87f-25041888489a
   Length: 49 characters ✓

5. Phoenix, AZ
   Session ID: eval-session-efb4e1d8-227a-4630-b0cc-be893fa5d06a
   Length: 49 characters ✓

Session ID Generation Summary

✓ Generated 5 unique session IDs
✓ All session IDs meet minimum length requirement (33+ chars)
✓ Session IDs stored in 'session_ids' dict and 'session_id_list' for retrieval
✓ All session IDs are unique (no duplicates)
✓ All session IDs are 33+ characters

Sessio

### Invoke Agent for Each Test Case

Now we'll invoke the deployed agent for each test case to generate trace data for evaluation. This cell:

1. **Loops through test cases**: Iterates over each city in our test dataset
2. **Generates queries**: Creates a standardized query asking about population and area
3. **Invokes the agent**: Calls `invoke_agent_runtime` API with the unique session ID
4. **Captures responses**: Stores the agent's response for each test case
5. **Displays progress**: Shows real-time progress and results

#### Query Format

Each query follows the format:
```
What is the population and area of {city}, {state}?
```

This standardized format ensures consistent agent behavior and enables meaningful evaluation comparisons.

#### Response Storage

Responses are stored in a dictionary (`agent_responses`) mapping city keys to their responses. This data structure enables:

- Easy lookup of responses by city
- Correlation with session IDs for span retrieval
- Analysis of response patterns across test cases

#### Error Handling

The invocation loop includes error handling to:
- Continue processing remaining test cases if one fails
- Log errors for debugging
- Track which invocations succeeded vs failed

In [38]:
# Invoke the agent for each test case
# This cell loops through all test cases, invokes the agent, and captures responses
#
# For each test case:
# 1. Generate a query asking about population and area
# 2. Invoke the agent using invoke_agent_runtime API
# 3. Capture and store the response
# 4. Display progress and results

import json
import time
import textwrap

# Dictionary to store agent responses for each test case
# Key: city_key (e.g., "New York, NY")
# Value: dict with query, response, session_id, and metadata
agent_responses = {}

# Track invocation statistics
successful_invocations = 0
failed_invocations = 0

print("Invoking Agent for Each Test Case")
print("="*60)
print(f"\nTotal test cases: {len(session_id_list)}")
print(f"Agent ARN: {citysearch_agent_arn}")
print("\n" + "="*60)

# Loop through each test case and invoke the agent
for i, test_case in enumerate(session_id_list):
    city = test_case['city']
    state = test_case['state']
    city_key = test_case['city_key']
    session_id = test_case['session_id']
    
    # Generate the query for this test case
    query = f"What is the population and area of {city}, {state}?"
    
    print(f"\n[{i + 1}/{len(session_id_list)}] {city_key}")
    print("-"*60)
    print(f"Query: {query}")
    print(f"Session ID: {session_id}")
    print("\nInvoking agent...")
    
    try:
        # Prepare the payload
        payload = json.dumps({"prompt": query})
        
        # Record start time for latency tracking
        start_time = time.time()
        
        # Invoke the agent using invoke_agent_runtime API
        response = agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=citysearch_agent_arn,
            runtimeSessionId=session_id,
            payload=payload,
            qualifier="DEFAULT"
        )
        
        # Calculate latency
        latency = time.time() - start_time
        
        # Read and parse the response
        response_body = response['response'].read()
        response_data = json.loads(response_body)
        
        # Store the response with metadata
        agent_responses[city_key] = {
            'query': query,
            'response': response_data,
            'session_id': session_id,
            'trace_id': response.get('traceId', 'N/A'),
            'latency_seconds': latency,
            'status': 'success'
        }
        
        successful_invocations += 1
        
        # Display the response (truncated for readability)
        response_str = str(response_data)
        if len(response_str) > 500:
            display_response = response_str[:500] + "..."
        else:
            display_response = response_str
        
        print(f"\n✓ Response received (latency: {latency:.2f}s)")
        print(f"\nResponse preview:")
        wrapped = textwrap.fill(display_response, width=70)
        print(wrapped)
        
        # Check for XML tags in response
        has_response_tag = "<response>" in response_str
        has_pop_tag = "<pop>" in response_str
        has_area_tag = "<area>" in response_str
        
        print(f"\nXML Tags: <response>={has_response_tag}, <pop>={has_pop_tag}, <area>={has_area_tag}")
        
    except Exception as e:
        # Handle invocation errors
        failed_invocations += 1
        
        # Store the error information
        agent_responses[city_key] = {
            'query': query,
            'response': None,
            'session_id': session_id,
            'error': str(e),
            'status': 'failed'
        }
        
        print(f"\n✗ Invocation failed: {str(e)}")
        print("Continuing with next test case...")

# Display summary
print("\n" + "="*60)
print("Invocation Summary")
print("="*60)
print(f"\nTotal test cases: {len(session_id_list)}")
print(f"Successful invocations: {successful_invocations}")
print(f"Failed invocations: {failed_invocations}")

if successful_invocations > 0:
    # Calculate average latency for successful invocations
    latencies = [r['latency_seconds'] for r in agent_responses.values() if r['status'] == 'success']
    avg_latency = sum(latencies) / len(latencies)
    print(f"Average latency: {avg_latency:.2f} seconds")

print("\n" + "="*60)
print("Results by Test Case")
print("="*60)
for city_key, result in agent_responses.items():
    status_icon = "✓" if result['status'] == 'success' else "✗"
    if result['status'] == 'success':
        print(f"\n{status_icon} {city_key}")
        print(f"   Session ID: {result['session_id']}")
        print(f"   Latency: {result['latency_seconds']:.2f}s")
    else:
        print(f"\n{status_icon} {city_key}")
        print(f"   Error: {result.get('error', 'Unknown error')}")

if successful_invocations == len(session_id_list):
    print(f"\n✓ All {successful_invocations} invocations completed successfully!")
    print("\nAgent responses are stored in 'agent_responses' dictionary.")
    print("Session IDs are stored in 'session_ids' dictionary for span retrieval.")
elif successful_invocations > 0:
    print(f"\n⚠ {successful_invocations} of {len(session_id_list)} invocations succeeded.")
    print("You may proceed with evaluation using the successful invocations.")
else:
    print(f"\n✗ All invocations failed. Please check the agent deployment and try again.")

Invoking Agent for Each Test Case

Total test cases: 5
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf


[1/5] New York, NY
------------------------------------------------------------
Query: What is the population and area of New York, NY?
Session ID: eval-session-5c8d51fb-d526-4d68-b531-f2af5d3c2331

Invoking agent...

✓ Response received (latency: 3.34s)

Response preview:
<thinking>I have retrieved several sources with data about the
population and area of New York, NY. I will now extract the most
recent and reliable information from these sources to provide an
accurate response.</thinking>  <response>New York City, often referred
to simply as New York or NYC, is a vibrant and diverse metropolis. As
of the most recent estimates, the city's population is approximately
8.48 million people, while the area spans about 302.6 square miles. A
fun fact about New York Cit...

XML Tags: <response>=True, <pop>=True, <area>=True

[2/5] Los Angeles, CA


### Wait for CloudWatch Log Propagation

After invoking the agent, we need to wait for the execution traces to propagate to CloudWatch Logs before we can retrieve them for evaluation.

#### Why Wait?

CloudWatch Logs operates with **eventual consistency**, meaning:

- **Log Ingestion Delay**: Logs are batched and sent to CloudWatch asynchronously
- **Index Propagation**: After ingestion, logs need time to be indexed and become queryable
- **Cross-Region Latency**: If using cross-region inference, additional latency may occur

#### Recommended Wait Time

A **30-second wait** is typically sufficient for:

| Factor | Typical Delay |
|--------|---------------|
| Log batching and transmission | 5-10 seconds |
| CloudWatch ingestion | 5-10 seconds |
| Index propagation | 5-15 seconds |
| **Total** | **15-35 seconds** |

#### What Happens During the Wait

While we wait, AgentCore Runtime is:

1. **Collecting Spans**: Gathering all execution trace data from the agent invocations
2. **Formatting Logs**: Converting spans to CloudWatch-compatible log format
3. **Transmitting Data**: Sending log batches to CloudWatch Logs service
4. **Indexing**: CloudWatch indexes the logs for efficient querying

After the wait completes, we'll be able to retrieve session spans using the CloudWatch Logs API.

## Section 5: AgentCore Evaluations

Now that we've invoked the agent and generated execution traces, we can run evaluations using the **AgentCore Evaluations API** via the starter toolkit.

### Using the Evaluation Class

The `bedrock-agentcore-starter-toolkit` provides an `Evaluation` class that simplifies the evaluation process by:

- **Automatic log retrieval**: Fetches session spans from CloudWatch internally
- **Built-in evaluators**: Access to pre-configured evaluators like `Builtin.Helpfulness` and `Builtin.ToolSelectionAccuracy`
- **Simple result access**: Clean methods to get successful and failed evaluation results

### Built-in Evaluators

| Evaluator | Description |
|-----------|-------------|
| `Builtin.Helpfulness` | Assesses whether the agent's response is helpful and addresses the user's query |
| `Builtin.ToolSelectionAccuracy` | Evaluates whether the agent selected appropriate tools for the task |

### Evaluation Output

Each evaluation returns:

- **value**: Numeric score (0.0 - 1.0) indicating evaluation result
- **label**: Categorical label (e.g., "PASS", "FAIL")
- **explanation**: Detailed reasoning for the evaluation result


In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation
import time

# Initialize the evaluation client
eval_client = Evaluation()

# Extract the runtime ID from the agent ARN
# ARN format: arn:aws:bedrock-agentcore:{region}:{account}:runtime/{runtime-id}
agent_id = citysearch_agent_arn.split('/')[-1]

print("Running AgentCore Evaluations")
print("=" * 60)
print(f"Agent ID: {agent_id}")
print(f"Sessions to evaluate: {len(session_ids)}")
print("=" * 60)

# Store all evaluation results
all_results = []

# Evaluate each session
for city_key, session_id in session_ids.items():
    print(f"\n[{city_key}]")
    print("-" * 60)
    print(f"Session ID: {session_id}")
    
    try:
        # Run evaluation using the starter toolkit
        results = eval_client.run(
            agent_id=agent_id,
            session_id=session_id,
            evaluators=["Builtin.Helpfulness", "Builtin.ToolSelectionAccuracy"]
        )
        
        # Get successful and failed results
        successful = results.get_successful_results()
        failed = results.get_failed_results()
        
        print(f"  Successful evaluations: {len(successful)}")
        print(f"  Failed evaluations: {len(failed)}")
        
        # Display results for each evaluator
        for result in successful:
            print(f"\n  📊 {result.evaluator_name}")
            print(f"     Score: {result.value:.2f}")
            print(f"     Label: {result.label}")
            if result.explanation:
                # Truncate long explanations
                explanation = result.explanation[:150] + "..." if len(result.explanation) > 150 else result.explanation
                print(f"     Explanation: {explanation}")
        
        # Store results
        all_results.append({
            'city_key': city_key,
            'session_id': session_id,
            'successful': successful,
            'failed': failed
        })
        
    except Exception as e:
        print(f"  ✗ Error: {str(e)}")
        all_results.append({
            'city_key': city_key,
            'session_id': session_id,
            'error': str(e)
        })

# Summary
print("\n" + "=" * 60)
print("Evaluation Summary")
print("=" * 60)

successful_evals = sum(1 for r in all_results if 'successful' in r and r['successful'])
print(f"Sessions evaluated successfully: {successful_evals}/{len(session_ids)}")

# Calculate average scores if we have results
helpfulness_scores = []
tool_accuracy_scores = []

for r in all_results:
    if 'successful' in r:
        for eval_result in r['successful']:
            if eval_result.evaluator_name == "Builtin.Helpfulness":
                helpfulness_scores.append(eval_result.value)
            elif eval_result.evaluator_name == "Builtin.ToolSelectionAccuracy":
                tool_accuracy_scores.append(eval_result.value)

if helpfulness_scores:
    print(f"\nAverage Helpfulness Score: {sum(helpfulness_scores)/len(helpfulness_scores):.2f}")
if tool_accuracy_scores:
    print(f"Average Tool Selection Accuracy: {sum(tool_accuracy_scores)/len(tool_accuracy_scores):.2f}")


Waiting 30 seconds for CloudWatch log propagation...
Done waiting.

Running AgentCore Evaluations
Agent ID: citysearch-U6S1xN3zuf
Sessions to evaluate: 5

[New York, NY]
------------------------------------------------------------
Session ID: eval-session-5c8d51fb-d526-4d68-b531-f2af5d3c2331


Evaluating session: eval-session-5c8d51fb-d526-4d68-b531-f2af5d3c2331

Mode: All traces (most recent 1000 spans)

Evaluators: Builtin.Helpfulness, Builtin.ToolSelectionAccuracy

/Users/justmul/Desktop/evals/.venv/lib/python3.13/site-packages/rich/live.py:260: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

  Successful evaluations: 2
  Failed evaluations: 0

  📊 Builtin.Helpfulness
     Score: 0.83
     Label: Very Helpful
     Explanation: The user's goal is straightforward: obtain the population and area of Houston, TX. The assistant's response directly addresses both pieces of informat...

  📊 Builtin.ToolSelectionAccuracy
     Score: 1.00
     Label: Yes
     Explanation: The user asked a straightforward factual question about Houston, TX's population and area. The assistant performed a web search with the query 'Housto...

[Phoenix, AZ]
------------------------------------------------------------
Session ID: eval-session-efb4e1d8-227a-4630-b0cc-be893fa5d06a


Evaluating session: eval-session-efb4e1d8-227a-4630-b0cc-be893fa5d06a

Mode: All traces (most recent 1000 spans)

Evaluators: Builtin.Helpfulness, Builtin.ToolSelectionAccuracy

  Successful evaluations: 5
  Failed evaluations: 0

  📊 Builtin.Helpfulness
     Score: 0.83
     Label: Very Helpful
     Explanation: The user's goal is straightforward: obtain the population and area of Phoenix, AZ. The assistant's response directly addresses this request by providi...

  📊 Builtin.ToolSelectionAccuracy
     Score: 1.00
     Label: Yes
     Explanation: The user asks a straightforward factual question about Phoenix, AZ's population and area. This is the first interaction in the conversation, and the a...

  📊 Builtin.ToolSelectionAccuracy
     Score: 1.00
     Label: Yes
     Explanation: The user asked for specific information about Phoenix, AZ: its population and area. The assistant performed a web search that returned several results...

  📊 Builtin.ToolSelectionAccuracy
     Score: 1.00
     Label: Yes
     Explanation: The user asked for the population and area of Phoenix, AZ. The assistant first performed a web search which returned several relevant results, 

## Section 6: Results Analysis

In this section, we'll analyze the evaluation results to understand our agent's performance:

1. **Calculate average scores**: Compute mean scores across all test cases
2. **Interpret results**: Understand what the scores mean
3. **Identify improvements**: Determine areas for agent enhancement

In [43]:
# Results Analysis - Updated for starter toolkit output
# Analyzes the evaluation results stored in all_results

import pandas as pd

print("Evaluation Results Analysis")
print("=" * 60)

# Build a list of result records for analysis
result_records = []
for r in all_results:
    if 'successful' in r and r['successful']:
        for eval_result in r['successful']:
            result_records.append({
                'City': r['city_key'],
                'Evaluator': eval_result.evaluator_name,
                'Score': eval_result.value,
                'Label': eval_result.label,
                'Explanation': eval_result.explanation[:100] + "..." if eval_result.explanation and len(eval_result.explanation) > 100 else eval_result.explanation
            })

if not result_records:
    print("\n⚠ No successful evaluation results to analyze.")
else:
    # Create DataFrame for analysis
    results_df = pd.DataFrame(result_records)
    
    print("\nAll Evaluation Results:")
    print("-" * 60)
    print(results_df[['City', 'Evaluator', 'Score', 'Label']].to_string(index=False))
    
    # Calculate averages by evaluator
    print("\n" + "=" * 60)
    print("Scores by Evaluator:")
    print("-" * 60)
    avg_by_evaluator = results_df.groupby('Evaluator')['Score'].agg(['mean', 'min', 'max', 'count'])
    avg_by_evaluator.columns = ['Average', 'Min', 'Max', 'Count']
    print(avg_by_evaluator.round(3).to_string())
    
    # Overall average
    overall_avg = results_df['Score'].mean()
    print(f"\n" + "=" * 60)
    print(f"Overall Average Score: {overall_avg:.3f}")
    print("=" * 60)
    
    # Interpretation
    print("\nScore Interpretation:")
    print("-" * 60)
    if overall_avg >= 0.9:
        print("✓ Excellent performance! The agent is working very well.")
    elif overall_avg >= 0.7:
        print("✓ Good performance with room for improvement.")
    elif overall_avg >= 0.5:
        print("⚠ Fair performance - consider refining the system prompt.")
    else:
        print("⚠ Performance needs improvement - review agent configuration.")


Evaluation Results Analysis

All Evaluation Results:
------------------------------------------------------------
           City                     Evaluator  Score        Label
   New York, NY           Builtin.Helpfulness   0.83 Very Helpful
   New York, NY Builtin.ToolSelectionAccuracy   1.00          Yes
Los Angeles, CA           Builtin.Helpfulness   0.83 Very Helpful
Los Angeles, CA Builtin.ToolSelectionAccuracy   1.00          Yes
    Chicago, IL           Builtin.Helpfulness   0.83 Very Helpful
    Chicago, IL Builtin.ToolSelectionAccuracy   1.00          Yes
    Chicago, IL Builtin.ToolSelectionAccuracy   0.00           No
    Chicago, IL Builtin.ToolSelectionAccuracy   1.00          Yes
    Houston, TX           Builtin.Helpfulness   0.83 Very Helpful
    Houston, TX Builtin.ToolSelectionAccuracy   1.00          Yes
    Phoenix, AZ           Builtin.Helpfulness   0.83 Very Helpful
    Phoenix, AZ Builtin.ToolSelectionAccuracy   1.00          Yes
    Phoenix, AZ Builtin.Tool

## Section 7: Cleanup

After completing the evaluation, it's important to clean up deployed resources to avoid unnecessary AWS charges. This section covers:

1. **Destroy the deployed agent**: Remove the agent from AgentCore Runtime
2. **Verify deletion**: Confirm the agent has been removed
3. **Additional cleanup reminders**: Resources that may need manual cleanup

### Important

Always run the cleanup cells when you're done with the workshop to avoid ongoing charges for:
- AgentCore Runtime compute resources
- ECR container storage
- CloudWatch log storage

In [ ]:
# Destroy the deployed agent
# This removes the agent from AgentCore Runtime
#
# WARNING: This action is irreversible. The agent will need to be
# re-deployed if you want to use it again.

print("Destroying Deployed Agent")
print("="*60)
print(f"\nAgent to destroy: {agent_name}")
print(f"Agent ARN: {citysearch_agent_arn}")
print("\n" + "="*60)

# Uncomment the following lines to actually destroy the agent:
# print("Destroying agent...")
# destroy_result = agentcore_runtime.destroy()
# print(f"\n✓ Agent destroyed successfully")
# print(f"\nDestroy result: {destroy_result}")

print("\n⚠ Agent destruction is commented out for safety.")
print("\nTo destroy the agent, uncomment the destroy lines above and re-run this cell.")
print("\nAlternatively, you can destroy from the command line:")
print(f"  agentcore destroy --agent-name {agent_name}")

🗑️ Destroying Bedrock AgentCore resources
Starting destroy operation for agent: citysearch (dry_run=False, delete_ecr_repo=False)


Destroying Deployed Agent

Agent to destroy: citysearch
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf



DEFAULT endpoint will be automatically deleted with agent
Deleted AgentCore agent: arn:aws:bedrock-agentcore:us-east-1:616455080294:runtime/citysearch-U6S1xN3zuf
Checking ECR repository: bedrock-agentcore-citysearch in region: us-east-1
Deleted 3 ECR images from bedrock-agentcore-citysearch
Deleted CodeBuild project: bedrock-agentcore-citysearch-builder
Deleted S3 artifact: citysearch/deployment.zip
Deleted S3 artifact: citysearch/source.zip
Deleted inline policy CodeBuildExecutionPolicy from role AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-672e22fb7a
Deleted CodeBuild IAM role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-672e22fb7a
Deleted IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-672e22fb7a
Removed agent configuration: citysearch
Cleared default agent (no agents remaining)
Removed configuration file: /Users/justmul/Desktop/evals/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-04-AgentCore-Runtime-Evals/.bedrock_agentcore.yaml
Destroy operation com


⚠ Agent destruction is commented out for safety.

To destroy the agent, uncomment the destroy lines above and re-run this cell.

Alternatively, you can destroy from the command line:
  agentcore destroy --agent-name citysearch


In [47]:
# Verify agent deletion
# This cell checks if the agent has been successfully removed

print("Verifying Agent Status")
print("="*60)

try:
    status_response = agentcore_runtime.status()
    status = status_response.endpoint.get('status', 'UNKNOWN')
    print(f"\nAgent status: {status}")
    
    if status == 'READY':
        print("\n⚠ Agent is still deployed.")
        print("  Run the destroy cell above to remove it.")
    elif status in ['DELETING', 'DELETED']:
        print("\n✓ Agent is being deleted or has been deleted.")
except Exception as e:
    if 'ResourceNotFoundException' in str(type(e).__name__) or 'not found' in str(e).lower():
        print("\n✓ Agent not found - deletion confirmed!")
    else:
        print(f"\nStatus check result: {str(e)}")
        print("\nThis may indicate the agent has been deleted.")

Configuration not found
Call .configure() first to set up your agent
Example: runtime.configure(entrypoint='my_agent.py')


Verifying Agent Status

Status check result: Must configure first. Call .configure() first.

This may indicate the agent has been deleted.


### Additional Cleanup Reminders

After destroying the agent, you may want to clean up these additional resources:

| Resource | Location | Action |
|----------|----------|--------|
| ECR Repository | AWS Console > ECR | Delete the container image repository |
| CloudWatch Logs | AWS Console > CloudWatch > Log Groups | Delete log groups starting with `/aws/bedrock-agentcore/` |
| IAM Roles | AWS Console > IAM > Roles | Delete auto-created execution roles |
| Local Files | This directory | Delete `citysearch.py`, `.bedrock_agentcore.yaml`, `Dockerfile` |

### Verify in AWS Console

To confirm all resources are cleaned up:

1. **AgentCore Runtime**: Check that no agents are listed
2. **ECR**: Verify the repository is deleted
3. **CloudWatch**: Confirm log groups are removed
4. **IAM**: Check for orphaned roles with `bedrock-agentcore` in the name

## Summary

Congratulations! You've completed the AgentCore Runtime Evaluations workshop module. In this notebook, you learned how to:

1. ✓ **Create a Strands Agent** with web search capabilities and XML output format
2. ✓ **Deploy to AgentCore Runtime** using the starter toolkit
3. ✓ **Invoke the deployed agent** via the `invoke_agent_runtime` API
4. ✓ **Retrieve session spans** from CloudWatch logs
5. ✓ **Run AgentCore Evaluations** with built-in evaluators
6. ✓ **Analyze evaluation results** to understand agent performance
7. ✓ **Clean up resources** to avoid unnecessary charges

### Key Takeaways

- **AgentCore Runtime** provides managed infrastructure for deploying AI agents with built-in observability
- **AgentCore Evaluations** offers standardized evaluation through the `evaluate()` API
- **Built-in evaluators** like `Builtin.Helpfulness` and `Builtin.ToolSelectionAccuracy` provide objective assessment
- **Session spans** from CloudWatch logs enable evaluation of real agent behavior

### Next Steps

- Explore additional built-in evaluators in the AgentCore Evaluations documentation
- Create custom evaluators for domain-specific assessment
- Integrate evaluations into your CI/CD pipeline for continuous agent quality monitoring
- Review the other workshop modules for more evaluation techniques